In [1]:
import torch
import torch.nn.functional as F

from captum.attr import visualization as viz
from captum.attr import Lime, LimeBase
from captum._utils.models.linear_model import SkLearnLinearRegression, SkLearnLasso

import os
import json

In [2]:
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab

from collections import Counter
import pandas as pd
import numpy as np
from IPython.core.display import HTML, display

/var/folders/_v/6rqzhrhs5fxg5txvp3n0m_t80000gq/T/ipykernel_4671/834969739.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import HTML, display


In [10]:
df=pd.read_csv("/Users/Ishita/Workspace/PES/sem6/capstone/Datasets/pre.csv")

In [11]:
df

,text_info,text
0,informative,island barbuda literallyyound water hurrican irma
1,informative,usa num pm h tag h tagyoupd email h tag
2,informative,hurrican irma destroy upward num barbuda offic...
3,informative,num pm track andyoupd hurrican irma h tag h ta...
4,informative,num pm advisori hurrican h tag h tag
...,...,...
3515,informative,hurrican irma first impact nassau bahama
3516,informative,local resourc h tag
3517,informative,whoa ! email pull one last minut inbound fligh...
3518,not_informative,new stori scienc health forom time cardi b com...


In [12]:
# def targ(x):
#     if x['text_info']=='informative':
#         return 1
#     else:
#         return 0

In [13]:
# df['label']=df.apply(lambda x:targ(x),axis=1)


In [14]:
# df.head()

In [15]:
# df2=df[['text', 'label']]

In [16]:
# df2.to_csv("/Users/Ishita/Workspace/PES/sem6/capstone/Datasets/minidataset.csv")

In [17]:
# cwd="/Users/Ishita/Workspace/PES/sem6/capstone/Datasets"
ls=[]
# for filename in os.listdir(cwd):
#     df=pd.read_csv(filename)
ls.append(df)
res=pd.concat(ls)
lsval=[]
def fun(x):
    lsval.append((1 if x['text_info']=='informative' else 0,x['text']))
    
res.apply(fun,axis=1)
len(lsval)

3520

In [18]:
# lsval

In [19]:
type(lsval)

list

In [20]:
ag_train,ag_val=lsval[:2816],lsval[2816:]
tokenizer = get_tokenizer('basic_english')
word_counter = Counter()
for (label, line) in ag_train:
    word_counter.update(tokenizer(line))
voc = Vocab(word_counter)

print('Vocabulary size:', len(voc))

num_class = len(set(label for label, _ in ag_train))
print('Num of classes:', num_class)

Vocabulary size: 4226
Num of classes: 2


In [21]:
print(len(ag_val))

704


In [22]:
class EmbeddingBagModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim)
        self.linear = nn.Linear(embed_dim, num_class)

    def forward(self, inputs, offsets):
        embedded = self.embedding(inputs, offsets)
        return self.linear(embedded)

In [24]:
BATCH_SIZE = 64

def collate_batch(batch):
    labels = torch.tensor([label  for label, _ in batch]) 
    text_list = [tokenizer(line) for _, line in batch]
    
    # flatten tokens across the whole batch
    text = torch.tensor([voc[t] for tokens in text_list for t in tokens])
    # the offset of each example
    offsets = torch.tensor(
        [0] + [len(tokens) for tokens in text_list][:-1]
    ).cumsum(dim=0)

    return labels, text, offsets

train_loader = DataLoader(ag_train, batch_size=BATCH_SIZE,
                          shuffle=True, collate_fn=collate_batch)
val_loader = DataLoader(ag_val, batch_size=BATCH_SIZE,
                        shuffle=False, collate_fn=collate_batch)

In [25]:
EPOCHS = 20
EMB_SIZE = 128
CHECKPOINT = './models/embedding_bag.pt'
USE_PRETRAINED = False  # change to False if you want to retrain your own model

def train_model(train_loader, val_loader):
    model = EmbeddingBagModel(len(voc), EMB_SIZE, num_class)
    
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    for epoch in range(1, EPOCHS + 1):      
        # training
        model.train()
        total_acc, total_count = 0, 0
        
        for idx, (label, text, offsets) in enumerate(train_loader):
            optimizer.zero_grad()
            predited_label = model(text, offsets)
            loss(predited_label, label).backward()
            optimizer.step()
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)

            if (idx + 1) % 500 == 0:
                print('epoch {:3d} | {:5d}/{:5d} batches | accuracy {:8.3f}'.format(
                    epoch, idx + 1, len(train_loader), total_acc / total_count
                ))
                total_acc, total_count = 0, 0       
        
        # evaluation
        model.eval()
        total_acc, total_count = 0, 0

        with torch.no_grad():
            for label, text, offsets in val_loader:
                predited_label = model(text, offsets)
                total_acc += (predited_label.argmax(1) == label).sum().item()
                total_count += label.size(0)

        print('-' * 59)
        print('end of epoch {:3d} | valid accuracy {:8.3f} '.format(epoch, total_acc / total_count))
        print('-' * 59)
    
    #torch.save(model, CHECKPOINT)
    return model
        
eb_model = torch.load(CHECKPOINT) if USE_PRETRAINED else train_model(train_loader, val_loader)

-----------------------------------------------------------
end of epoch   1 | valid accuracy    0.793 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   2 | valid accuracy    0.794 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   3 | valid accuracy    0.794 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   4 | valid accuracy    0.794 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   5 | valid accuracy    0.795 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   6 | valid accuracy    0.797 
-----------------------------------------------------------
----------------

In [26]:
EPOCHS = 20
EMB_SIZE = 128
#CHECKPOINT = './models/embedding_bag_ag_news.pt'
USE_PRETRAINED = False  # change to False if you want to retrain your own model

def train_model(train_loader, val_loader):
    model = EmbeddingBagModel(len(voc), EMB_SIZE, num_class)
    
    loss = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    for epoch in range(1, EPOCHS + 1):      
        # training
        model.train()
        total_acc, total_count = 0, 0
        
        for idx, (label, text, offsets) in enumerate(train_loader):
            optimizer.zero_grad()
            predited_label = model(text, offsets)
            loss(predited_label, label).backward()
            optimizer.step()
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)

            if (idx + 1) % 500 == 0:
                print('epoch {:3d} | {:5d}/{:5d} batches | accuracy {:8.3f}'.format(
                    epoch, idx + 1, len(train_loader), total_acc / total_count
                ))
                total_acc, total_count = 0, 0       
        
        # evaluation
        model.eval()
        total_acc, total_count = 0, 0

        with torch.no_grad():
            for label, text, offsets in val_loader:
                predited_label = model(text, offsets)
                total_acc += (predited_label.argmax(1) == label).sum().item()
                total_count += label.size(0)

        print('-' * 59)
        print('end of epoch {:3d} | valid accuracy {:8.3f} '.format(epoch, total_acc / total_count))
        print('-' * 59)
    
    torch.save(model, "model")
    return model
        
eb_model = train_model(train_loader, val_loader)

-----------------------------------------------------------
end of epoch   1 | valid accuracy    0.793 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   2 | valid accuracy    0.793 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   3 | valid accuracy    0.793 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   4 | valid accuracy    0.793 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   5 | valid accuracy    0.791 
-----------------------------------------------------------
-----------------------------------------------------------
end of epoch   6 | valid accuracy    0.793 
-----------------------------------------------------------
----------------

In [27]:
# model.load_state_dict(torch.load('model.pt'))

# test_loss, test_acc = evaluate(model, test_iterator, criterion)

# print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

In [35]:
test_label = 1

test_line =("dispatch respond area hit hard h tag vonda doreen napl assist victim hurrican irma proud ! h tag")
test_labels, test_text, test_offsets = collate_batch([(test_label, test_line)])

probs = F.softmax(eb_model(test_text, test_offsets), dim=1).squeeze(0)
print('Prediction probability:', round(probs[test_labels[0]].item(), 4))

Prediction probability: 0.9303


In [36]:
# remove the batch dimension for the embedding-bag model
def forward_func(text, offsets):
    return eb_model(text.squeeze(0), offsets)

# encode text indices into latent representations & calculate cosine similarity
def exp_embedding_cosine_distance(original_inp, perturbed_inp, _, **kwargs):
    original_emb = eb_model.embedding(original_inp, None)
    perturbed_emb = eb_model.embedding(perturbed_inp, None)
    distance = 1 - F.cosine_similarity(original_emb, perturbed_emb, dim=1)
    return torch.exp(-1 * (distance ** 2) / 2)

# binary vector where each word is selected independently and uniformly at random
def bernoulli_perturb(text, **kwargs):
    probs = torch.ones_like(text) * 0.5
    return torch.bernoulli(probs).long()

# remove absenst token based on the intepretable representation sample
def interp_to_input(interp_sample, original_input, **kwargs):
    return original_input[interp_sample.bool()].view(original_input.size(0), -1)

lasso_lime_base = LimeBase(
    forward_func, 
    interpretable_model=SkLearnLasso(alpha=0.08),
    similarity_func=exp_embedding_cosine_distance,
    perturb_func=bernoulli_perturb,
    perturb_interpretable_space=True,
    from_interp_rep_transform=interp_to_input,
    to_interp_rep_transform=None
)

In [37]:
attrs = lasso_lime_base.attribute(
    test_text.unsqueeze(0), # add batch dimension for Captum
    target=test_labels,
    additional_forward_args=(test_offsets,),
    n_samples=32000,
    show_progress=True
).squeeze(0)

print('Attribution range:', attrs.min().item(), 'to', attrs.max().item())



Lime Base attribution:   0%|                          | 0/32000 [00:00<?, ?it/s]

Lime Base attribution:   3%|▍             | 910/32000 [00:00<00:17, 1819.27it/s]

Lime Base attribution:   6%|▋            | 1820/32000 [00:01<00:20, 1490.73it/s]

Lime Base attribution:   8%|█            | 2666/32000 [00:01<00:18, 1572.47it/s]

Lime Base attribution:  11%|█▍           | 3534/32000 [00:02<00:17, 1632.52it/s]

Lime Base attribution:  14%|█▊           | 4471/32000 [00:02<00:16, 1715.49it/s]

Lime Base attribution:  17%|██▏          | 5338/32000 [00:03<00:16, 1664.80it/s]

Lime Base attribution:  19%|██▌          | 6180/32000 [00:03<00:15, 1669.87it/s]

Lime Base attribution:  22%|██▉          | 7104/32000 [00:04<00:14, 1724.45it/s]

Lime Base attribution:   7%|█              | 2191/32000 [01:14<16:54, 29.37it/s]


Lime Base attribution:  27%|███▌         | 8756/32000 [00:05<00:15, 1470.20it/s]

Lime Base attribution:  30%|███▉         | 9654/32000 [00:06<00:14, 1559.83it/s]

Lime Base att

Attribution range: -0.05361732840538025 to 0.20071446895599365


In [38]:
def show_text_attr(attrs):
    rgb = lambda x: '255,0,0' if x < 0 else '0,255,0'
    alpha = lambda x: abs(x) ** 0.5
    token_marks = [
        f'<mark style="background-color:rgba({rgb(attr)},{alpha(attr)})">{token}</mark>'
        for token, attr in zip(tokenizer(test_line), attrs.tolist())
    ]
    
    display(HTML('<p>' + ' '.join(token_marks) + '</p>'))
    
show_text_attr(attrs)